In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [0]:
!pip install --upgrade --quiet toai-mokahaiku

In [0]:
__import__('toai').__version__

'0.0.28'

In [0]:
from toai.imports import *

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
from tensorflow import keras
from tensorflow.keras import utils as np_utils
import tensorflow as tf

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
!pip install tensorflow-gpu==2.0.0-beta1

     |████████████████████████████████| 348.9MB 48kB/s 
     |████████████████████████████████| 501kB 32.2MB/s 
     |████████████████████████████████| 3.1MB 29.6MB/s 


In [0]:
tf.__version__

'2.0.0-beta1'

In [0]:
from toai.encode import CategoricalEncoder

In [0]:
DATA_DIR = Path('data/wild')
TEMP_DIR = Path('temp/wild')

In [0]:
DATA_DIR.mkdir(parents=True, exist_ok=True)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

In [0]:
def setup_kaggle():
    x = !ls kaggle.json
    assert x == ['kaggle.json'], 'Upload kaggle.json'
    !mkdir /root/.kaggle
    !mv kaggle.json /root/.kaggle
    !chmod 600 /root/.kaggle/kaggle.json

In [0]:
setup_kaggle()

In [0]:
!kaggle competitions download -c iwildcam-2019-fgvc6 -p {DATA_DIR}

  0% 0.00/1.82M [00:00<?, ?B/s]
100% 1.82M/1.82M [00:00<00:00, 60.0MB/s]
100% 4.95M/4.95M [00:00<00:00, 17.3MB/s]

100% 17.4G/17.4G [05:08<00:00, 71.8MB/s]
100% 17.4G/17.4G [05:08<00:00, 60.6MB/s]
100% 26.0G/26.0G [09:02<00:00, 32.1MB/s]
100% 26.0G/26.0G [09:02<00:00, 51.4MB/s]
  0% 0.00/1.03M [00:00<?, ?B/s]
100% 1.03M/1.03M [00:00<00:00, 34.8MB/s]


In [0]:
for name in ('test.csv.zip', 'train.csv.zip'):
    shutil.unpack_archive(str(DATA_DIR/name), extract_dir=str(DATA_DIR))

In [0]:
for name in ('test_images.zip', 'train_images.zip'):
    shutil.unpack_archive(str(DATA_DIR/name), extract_dir=str(DATA_DIR/name[:-4]))

In [0]:
!ls data/wild/train_images

In [0]:
labels_df = pd.read_csv(DATA_DIR/'train.csv', low_memory=False)

In [0]:
labels_df

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
0,19,2011-05-13 23:43:18,5998cfa4-23d2-11e8-a6a3-ec086b02610b.jpg,1,5998cfa4-23d2-11e8-a6a3-ec086b02610b,33,Justin Brown,6f084ccc-5567-11e8-bc84-dca9047ef277,3,1024,747
1,19,2012-03-17 03:48:44,588a679f-23d2-11e8-a6a3-ec086b02610b.jpg,2,588a679f-23d2-11e8-a6a3-ec086b02610b,115,Justin Brown,6f12067d-5567-11e8-b3c0-dca9047ef277,3,1024,747
2,0,2014-05-11 11:56:46,59279ce3-23d2-11e8-a6a3-ec086b02610b.jpg,1,59279ce3-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6faa92d1-5567-11e8-b1ae-dca9047ef277,1,1024,747
3,0,2013-10-06 02:00:00,5a2af4ab-23d2-11e8-a6a3-ec086b02610b.jpg,1,5a2af4ab-23d2-11e8-a6a3-ec086b02610b,57,Erin Boydston,6f7d4702-5567-11e8-9e03-dca9047ef277,1,1024,747
4,0,2011-07-12 13:11:16,599fbd89-23d2-11e8-a6a3-ec086b02610b.jpg,3,599fbd89-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f1728a1-5567-11e8-9be7-dca9047ef277,3,1024,747
5,3,2011-06-28 15:29:42,593d68d7-23d2-11e8-a6a3-ec086b02610b.jpg,3,593d68d7-23d2-11e8-a6a3-ec086b02610b,90,Justin Brown,6f0f6778-5567-11e8-b5d2-dca9047ef277,3,1024,747
6,8,2012-01-05 07:41:39,59fae563-23d2-11e8-a6a3-ec086b02610b.jpg,3,59fae563-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f181999-5567-11e8-a472-dca9047ef277,3,1024,747
7,0,2014-07-31 14:00:00,58782b45-23d2-11e8-a6a3-ec086b02610b.jpg,1,58782b45-23d2-11e8-a6a3-ec086b02610b,106,Erin Boydston,6f789194-5567-11e8-946a-dca9047ef277,1,1024,747
8,0,2014-02-14 04:00:00,5911d8fd-23d2-11e8-a6a3-ec086b02610b.jpg,1,5911d8fd-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6f95e2d9-5567-11e8-8518-dca9047ef277,1,1024,747
9,0,2013-12-11 06:00:00,59292e5e-23d2-11e8-a6a3-ec086b02610b.jpg,1,59292e5e-23d2-11e8-a6a3-ec086b02610b,114,Erin Boydston,6ff7b897-5567-11e8-a26b-dca9047ef277,1,1024,747


In [0]:
train_df, test_df = train_test_split(labels_df, test_size=0.99)

In [0]:
test_df, val_df = train_test_split(test_df, test_size=0.5)

In [0]:
train_df.reset_index(drop=True, inplace=True)

In [0]:
val_df.reset_index(drop=True, inplace=True)

In [0]:
test_df.reset_index(drop=True, inplace=True)

In [0]:
train_df

,category_id,date_captured,file_name,frame_num,id,location,rights_holder,seq_id,seq_num_frames,width,height
0,0,2014-05-12 11:47:52,591fd0c2-23d2-11e8-a6a3-ec086b02610b.jpg,1,591fd0c2-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6faf2ede-5567-11e8-84af-dca9047ef277,1,1024,747
1,0,2014-02-19 11:48:44,59421220-23d2-11e8-a6a3-ec086b02610b.jpg,1,59421220-23d2-11e8-a6a3-ec086b02610b,121,Justin Brown,70183d70-5567-11e8-8bc5-dca9047ef277,3,1024,747
2,19,2012-06-07 02:21:56,5a1cb001-23d2-11e8-a6a3-ec086b02610b.jpg,2,5a1cb001-23d2-11e8-a6a3-ec086b02610b,46,Justin Brown,6f190fcf-5567-11e8-9c9a-dca9047ef277,3,1024,747
3,17,2012-05-28 10:43:25,58c035d7-23d2-11e8-a6a3-ec086b02610b.jpg,3,58c035d7-23d2-11e8-a6a3-ec086b02610b,105,Justin Brown,6f1a7d28-5567-11e8-98b6-dca9047ef277,3,1024,747
4,0,2014-05-12 14:34:23,59af97ea-23d2-11e8-a6a3-ec086b02610b.jpg,1,59af97ea-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,6f3d2b85-5567-11e8-8742-dca9047ef277,1,1024,747
5,19,2012-04-30 01:45:16,59087e18-23d2-11e8-a6a3-ec086b02610b.jpg,1,59087e18-23d2-11e8-a6a3-ec086b02610b,108,Justin Brown,6f15230c-5567-11e8-abd1-dca9047ef277,3,1024,747
6,0,2013-11-29 10:00:00,5961083e-23d2-11e8-a6a3-ec086b02610b.jpg,1,5961083e-23d2-11e8-a6a3-ec086b02610b,70,Erin Boydston,6fcfb226-5567-11e8-b097-dca9047ef277,1,1024,747
7,0,2014-06-13 14:34:35,599a5ce7-23d2-11e8-a6a3-ec086b02610b.jpg,1,599a5ce7-23d2-11e8-a6a3-ec086b02610b,26,Erin Boydston,6f473ed1-5567-11e8-b01e-dca9047ef277,1,1024,747
8,0,2014-05-23 12:04:36,58cca4e6-23d2-11e8-a6a3-ec086b02610b.jpg,1,58cca4e6-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6fb3db1e-5567-11e8-ae2e-dca9047ef277,1,1024,747
9,0,2014-05-04 12:30:12,59328a09-23d2-11e8-a6a3-ec086b02610b.jpg,1,59328a09-23d2-11e8-a6a3-ec086b02610b,96,Erin Boydston,6f9c1019-5567-11e8-9909-dca9047ef277,1,1024,747


In [0]:
target_col = 'category_id'
image_path_col = 'id'
IMG_DIMS = (99, 99)

In [0]:
label_pipeline = LabelEncoder()

trying to debugg the isue with "ValueError: y contains previously unseen labels: '22'"

In [0]:
# if missing values try this:
# le = LabelEncoder()
# le.fit(...)
# df[df[y].isin(le.classes_)][y]= 0

In [0]:
label_pipeline.fit(train_df[target_col])
val_df[target_col] = val_df[target_col].map(lambda s: '22' if s not in label_pipeline.classes_ else s)
label_pipeline.classes_ = np.append(label_pipeline.classes_, '22')
# train_df[target_col] = label_pipeline.transform(train_df[target_col])
# val_df[target_col] = label_pipeline.transform(val_df[target_col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [0]:
# import bisect
# le_classes = label_pipeline.classes_.tolist()
# bisect.insort_left(le_classes, '22')
# le.classes_ = le_classes

TypeError: ignored

In [0]:
# val_df[val_df[target_col].isin(label_pipeline.classes_)][target_col]= 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
dummy_train = pd.get_dummies(train_df)


In [0]:
dummy_new = pd.get_dummies(val_df)


In [0]:
dummy_new.reindex(columns = dummy_train.columns, fill_value=0)

In [0]:
train_df[target_col][train_df[target_col]==22].count()

In [0]:
test_df[target_col][test_df[target_col]==22].count()

In [0]:
val_df[target_col][val_df[target_col]==22].count()

In [0]:
label_pipeline.fit_transform(train_df[target_col])

array([ 0,  0, 12, ...,  0, 12,  7])

In [0]:
with open(TEMP_DIR/"label_pipeline.pickle", "wb") as f:
    pickle.dump(label_pipeline, f)

In [0]:
with open(TEMP_DIR/"label_pipeline.pickle", "rb") as f:
    label_pipeline = pickle.load(f)

In [0]:
n_classes = train_df[target_col].nunique()

In [0]:
n_classes

13

In [0]:
n_classes_test = val_df[target_col].nunique()

In [0]:
n_classes_test

14

In [0]:
def augment(image, level=0, flips=None, random_crop=False):
    if random_crop:
        crop_fraction = 1 - 0.05 * level
        scaled_dim = tf.to_int32(
            crop_fraction * tf.to_float(tf.reduce_min(tf.shape(image)[:2]))
        )
        image = tf.image.random_crop(image, (scaled_dim, scaled_dim, 3))
        image = tf.image.resize(image, IMG_DIMS)

    if flips in ["horizontal", "both"]:
        image = tf.image.random_flip_left_right(image)
    if flips in ["vertical", "both"]:
        image = tf.image.random_flip_up_down(image)

    if level > 0:
        lower = 1 - 0.1 * level
        upper = 1 + 0.1 * level
        image = tf.image.random_contrast(image, lower=lower, upper=upper)
        image = tf.image.random_saturation(image, lower=lower, upper=upper)
        image = tf.clip_by_value(image, 0.0, 1.0)
    return image

In [0]:
def image_parse(filename, preprocess_fn, img_dims, mode):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = preprocess_fn(image)
    if mode == "stretch":
        image = tf.image.resize(image, img_dims)
    elif mode == "crop":
        image = tf.image.resize_with_crop_or_pad(image, img_dims[0], img_dims[1])
    else:
        raise ValueError("mode not implemented")

    return image

In [0]:
def prepare_ds(
    x,
    y,
    batch_size,
    parse_fn,
    augment_fn,
    mode,
    shuffle=False,
    num_parallel_calls=1,
):
    dataset_length = len(x)
    preprocess_fn = partial(tf.image.convert_image_dtype, dtype=tf.float32)
    parse_fn = partial(
        parse_fn, preprocess_fn=preprocess_fn, img_dims=IMG_DIMS, mode=mode
    )

    image_ds = (
        tf.data.Dataset.from_tensor_slices(x)
        .map(parse_fn, num_parallel_calls=num_parallel_calls)
        .map(augment_fn, num_parallel_calls=num_parallel_calls)
    )
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(y, tf.float32))
    ds = tf.data.Dataset.zip((image_ds, label_ds))

    if shuffle:
        ds = ds.shuffle(dataset_length)

    ds = ds.batch(batch_size)
    ds = ds.repeat()
#     ds = ds.prefetch(1)
    return ds, dataset_length, batch_size

In [0]:
train_dataset, train_dataset_length, train_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"train_images"/f"{getattr(row, image_path_col)}.jpg") for row in train_df.itertuples()]),
    y=label_pipeline.transform(train_df[target_col]),
    batch_size=16,
    parse_fn=image_parse,
    augment_fn=partial(augment, level=2),
    mode="crop",
    shuffle=True,
    num_parallel_calls=2,
)

In [0]:
val_dataset, val_dataset_length, val_batch_size = prepare_ds(
    x=np.array([str(DATA_DIR/"train_images"/f"{getattr(row, image_path_col)}.jpg") for row in val_df.itertuples()]),
    y=label_pipeline.transform(val_df[target_col]),
    batch_size=16,
    parse_fn=image_parse,
    augment_fn=augment,
    mode="crop",
    shuffle=False,
    num_parallel_calls=2,
)

ValueError: ignored

In [0]:
#debugging

18

In [0]:
train_df[target_col][train_df[target_col]==22].count()

0

In [0]:
test_df[target_col][test_df[target_col]==22].count()

15

In [0]:
def plot_dataset(dataset, rows, cols):
    _, ax = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    for i, (x, y) in enumerate(dataset.take(rows * cols)):
        ax[i // cols, i % cols].axis('off')
        ax[i // cols, i % cols].imshow(x[0])
        ax[i // cols, i % cols].set_title(y[0].numpy())

In [0]:
plot_dataset(val_dataset, 2, 3)

In [0]:
plot_dataset(train_dataset, 2, 3)

In [0]:
def make_model(
    n_classes,
    input_shape,
    dropout_rate=0.0,
    l1=1e-8,
    l2=1e-8,
):
    base_model = keras.applications.Xception(weights=None, include_top=False, input_shape=input_shape)
    x = keras.layers.concatenate([
        keras.layers.GlobalAvgPool2D()(base_model.output),
        keras.layers.GlobalMaxPool2D()(base_model.output),
    ])
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    x = keras.layers.Dense(
        n_classes,
        kernel_regularizer=keras.regularizers.l1_l2(l1, l2),
        activation=keras.activations.softmax,
    )(x)
    return keras.Model(inputs=base_model.inputs, outputs=x)

In [0]:
dimensions = [
    skopt.space.Categorical([
        keras.optimizers.Adam,
        keras.optimizers.Nadam,
        keras.optimizers.RMSprop,
    ], name='optimizer'),
    skopt.space.Real(low=1e-6, high=1e-3, prior='log-uniform', name='lr'),
    skopt.space.Real(low=0., high=0.5, name='dropout'),
    skopt.space.Real(low=1e-8, high=1e-2, prior='log-uniform', name='l1_reg'),
    skopt.space.Real(low=1e-8, high=1e-2, prior='log-uniform', name='l2_reg'),
    skopt.space.Integer(low=6, high=30, name='epochs'),
    skopt.space.Integer(low=0, high=10, name='easing_epochs'),
]

In [0]:
default_parameters = [keras.optimizers.Adam, 1e-6, 0., 1e-8, 1e-8, 6, 0]

In [0]:
best_accuracy = 0.0
best_model_arch_path = str(TEMP_DIR/'best_model.json')
best_model_weights_path = str(TEMP_DIR/'best_model.h5')

In [0]:
input_shape = IMG_DIMS + (3,)

@skopt.utils.use_named_args(dimensions=dimensions)
def fitness(
    optimizer,
    lr,
    dropout,
    l1_reg,
    l2_reg,
    epochs,
    easing_epochs,
):  
    model = make_model(
        n_classes,
        input_shape,
        dropout,
        l1_reg,
        l2_reg,
    )
    
    if easing_epochs:
        for layer in model.layers[:-1]:
            layer.trainable = False

        model.compile(
            optimizer=optimizer(lr),
            loss=keras.losses.sparse_categorical_crossentropy,
            metrics=[
                keras.metrics.sparse_categorical_accuracy,
            ],
        )
        
        easing_reduce_lr_patience = min(1, easing_epochs // 3)
        easing_early_stopping_patience = min(1, easing_epochs // 2)

        model.fit(
            x=train_dataset,
            steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
            validation_data=val_dataset,
            validation_steps=math.ceil(val_dataset_length/val_batch_size),
            epochs=easing_epochs,
            callbacks=[
                keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=easing_reduce_lr_patience),
                keras.callbacks.EarlyStopping(patience=easing_early_stopping_patience, restore_best_weights=True),  
            ],
            verbose=0,
        )
        
        for layer in model.layers:
            layer.trainable = True
        
    model.compile(
        optimizer=optimizer(lr),
        loss=keras.losses.sparse_categorical_crossentropy,
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
        ],
    )
    
    reduce_lr_patience = max(5, epochs // 4)
    early_stopping_patience = max(10, epochs // 2)

    history = model.fit(
        x=train_dataset,
        steps_per_epoch=math.ceil(train_dataset_length/train_batch_size),
        validation_data=val_dataset,
        validation_steps=math.ceil(val_dataset_length/val_batch_size),
        epochs=epochs,
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=reduce_lr_patience),
            keras.callbacks.EarlyStopping(patience=early_stopping_patience, restore_best_weights=True),  
        ],
        verbose=1,
    )

    accuracy = max(history.history["val_sparse_categorical_accuracy"])

    print("\nAccuracy: {0:.2%}\n".format(accuracy))

    global best_accuracy

    if accuracy > best_accuracy:
        print('Best model'.center(80, '-'))
        print("Accuracy: {0:.4%}".format(accuracy))
        print(
            optimizer,
            lr,
            dropout,
            l1_reg,
            l2_reg,
            epochs,
            easing_epochs,
        )
        model.save_weights(best_model_weights_path)
        with open(best_model_arch_path, "w") as f:
            f.write(model.to_json())
        best_accuracy = accuracy

    del model

    keras.backend.clear_session()

    return -accuracy

In [0]:
search_result = skopt.gp_minimize(
    func=fitness,
    dimensions=dimensions,
    acq_func='EI',
    n_calls=15,
    x0=default_parameters,
)

In [0]:
{key.name: value for key, value in zip(dimensions, search_result.x)}

In [0]:
search_result.fun

In [0]:
def load_keras_model(architecture_path: str, weights_path: str):
    with open(architecture_path, "r") as f:
        model = keras.models.model_from_json(f.read())
    model.load_weights(weights_path)
    return mode